In [ ]:
import pandas as pd
import numpy as np

# import matplotlib.pyplot as plt
import yaml
import os
import glob
import sys
sys.path.append('../')
from utils import *

BASE_URL = 'C:/Users/Admin/Desktop/DATN/DATA/SVIC-APT-2021/'


In [ ]:
# file_paths = [f"{BASE_URL}phase{i}.csv" for i in range(1, 5)]

# df_temp = [pd.read_csv(file_path) for file_path in file_paths]
# df = pd.concat(df_temp, ignore_index=True)

In [ ]:
# df = df.dropna(axis=1, how='all')
df = pd.read_csv(BASE_URL+'Training.csv')
df2 = pd.read_csv(BASE_URL+'Testing.csv')

In [ ]:
df.value_counts('Label')

In [ ]:
df2.value_counts('Label')

In [ ]:
list_features = ['Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWR Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg',
       'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg', 'Fwd Bulk Rate Avg',
       'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min', 'Label']

df = df[list_features]
df2 = df2[list_features]


In [ ]:
cols_unique_val = []
cols_unique_val2 = []

# Dropping features contain unique value for every instance
for i in range(1,2):
    for col in df.columns:
        if df[col].nunique() == i:
            cols_unique_val.append(col)

for i in range(1,2):
    for col in df.columns:
        if df2[col].nunique() == i:
            cols_unique_val2.append(col)

print(f'Features contain unique value for every instance: {cols_unique_val}')
print(f'Features contain unique value for every instance: {cols_unique_val2}')

print(len(cols_unique_val))
print(len(cols_unique_val2))

df.drop(cols_unique_val, axis=1, inplace=True)
df2.drop(cols_unique_val, axis=1, inplace=True)


In [ ]:
df2

In [ ]:
df2.value_counts('Label')

In [ ]:
stage_mapping = {
    'NormalTraffic': 0,        
    'Reconnaissance': 1,     
    'InitialCompromise': 1,
    'LateralMovement': 2,
    'Pivoting': 2,
    'DataExfiltration': 3
}

df['Label'] = df['Label'].map(stage_mapping)
df2['Label'] = df2['Label'].map(stage_mapping)

In [ ]:
df.value_counts('Label')

In [ ]:
# df.value_counts('Stage')
df.replace([-np.inf, np.inf], np.nan, inplace=True)
df.dropna(axis=0, inplace=True)

# Dropping duplicate samples
df.drop_duplicates(inplace=True)

# df.value_counts('Stage')
df2.replace([-np.inf, np.inf], np.nan, inplace=True)
df2.dropna(axis=0, inplace=True)

# Dropping duplicate samples
df2.drop_duplicates(inplace=True)

In [ ]:
df.to_csv(BASE_URL + 'scvic_training.csv')
df2.to_csv(BASE_URL + 'scvic_test.csv')

-----------------------
-----------------------
-----------------------


In [ ]:
df = pd.read_csv(BASE_URL + 'scvic_training.csv')
df2 = pd.read_csv(BASE_URL + 'scvic_test.csv')

<h2> Training </h2>


In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df2.drop('Unnamed: 0', axis=1, inplace=True)


In [ ]:
df2.columns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
import xgboost as xgb


from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline, make_pipeline

In [ ]:
x_train = df.iloc[:, :-1].values
y_train = df.iloc[:, -1].values

x_test = df2.iloc[:, :-1].values
y_test = df2.iloc[:, -1].values


smote = SMOTE(random_state=42)

x_train, y_train = smote.fit_resample(x_train, y_train)
x_test, y_test = smote.fit_resample(x_test, y_test)


In [ ]:
df.value_counts('Label')

RANDOM FOREST

In [ ]:
rf_scvic = RandomForestClassifier(
    n_estimators=100,
    n_jobs=12,
)

In [ ]:
rf_scvic.fit(x_train, y_train)

# from sklearn.model_selection import KFold, cross_val_score


# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# # Perform cross-validation
# scores = cross_val_score(rf_scvic, x_train, y_train, cv=kf, scoring='accuracy')

# print("Cross-Validation Scores:", scores)
# print("Mean Accuracy:", scores.mean())
# print("Standard Deviation:", scores.std())

In [ ]:
y_pred = rf_scvic.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)

f1 = f1_score(y_test, y_pred, average='weighted')
f2 = f1_score(y_test, y_pred, average='macro')
f3 = f1_score(y_test, y_pred, average='micro')
# accuracy3 = f1_score(y_test, y_pred, average='binary')
pr1 = precision_score(y_test, y_pred, average='weighted')
pr2 = precision_score(y_test, y_pred, average='macro')
pr3 = precision_score(y_test, y_pred, average='micro')

re1 = recall_score(y_test, y_pred, average='weighted')
re2 = recall_score(y_test, y_pred, average='macro')
re3 = recall_score(y_test, y_pred, average='micro')


print(accuracy)
print("-------------------------------------------------------")
print("f1 : " ,f1)
print("f1 : " ,f2)
print("f1 : " ,f3)
print("-------------------------------------------------------")
print("precision : " , pr1)
print("precision : " , pr2)
print("precision : " , pr3)
print("-------------------------------------------------------")
print("recall : " , re1)
print("recall : " , re2)
print("recall : " , re3)



In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

LOGISTIC REGRESSION

In [ ]:
log_scvic = LogisticRegression(
    C=0.1,
    solver='sag',
    max_iter=200,
    multi_class='multinomial'
)

In [ ]:
log_scvic.fit(x_train, y_train)

In [ ]:
y_pred = log_scvic.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

f1 = f1_score(y_test, y_pred, average='weighted')
f2 = f1_score(y_test, y_pred, average='macro')
f3 = f1_score(y_test, y_pred, average='micro')
# accuracy3 = f1_score(y_test, y_pred, average='binary')
pr1 = precision_score(y_test, y_pred, average='weighted')
pr2 = precision_score(y_test, y_pred, average='macro')
pr3 = precision_score(y_test, y_pred, average='micro')

re1 = recall_score(y_test, y_pred, average='weighted')
re2 = recall_score(y_test, y_pred, average='macro')
re3 = recall_score(y_test, y_pred, average='micro')


print(accuracy)
print("-------------------------------------------------------")
print("f1 : " ,f1)
print("f1 : " ,f2)
print("f1 : " ,f3)
print("-------------------------------------------------------")
print("precision : " , pr1)
print("precision : " , pr2)
print("precision : " , pr3)
print("-------------------------------------------------------")
print("recall : " , re1)
print("recall : " , re2)
print("recall : " , re3)



XGBOOST

In [ ]:
xgb_scvic = xgb.XGBClassifier(objective='multi:softmax')

In [ ]:
xgb_scvic.fit(x_train, y_train)

In [ ]:
y_pred = xgb_scvic.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

In [ ]:
f1 = f1_score(y_test, y_pred, average='weighted')
f2 = f1_score(y_test, y_pred, average='macro')
f3 = f1_score(y_test, y_pred, average='micro')
# accuracy3 = f1_score(y_test, y_pred, average='binary')
pr1 = precision_score(y_test, y_pred, average='weighted')
pr2 = precision_score(y_test, y_pred, average='macro')
pr3 = precision_score(y_test, y_pred, average='micro')

re1 = recall_score(y_test, y_pred, average='weighted')
re2 = recall_score(y_test, y_pred, average='macro')
re3 = recall_score(y_test, y_pred, average='micro')


print(accuracy)
print("-------------------------------------------------------")
print("f1 : " ,f1)
print("f1 : " ,f2)
print("f1 : " ,f3)
print("-------------------------------------------------------")
print("precision : " , pr1)
print("precision : " , pr2)
print("precision : " , pr3)
print("-------------------------------------------------------")
print("recall : " , re1)
print("recall : " , re2)
print("recall : " , re3)

conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

PREDICT


In [ ]:
df_zz = df2[df2['Label'] == 3]
df_zz.iloc[0]
t1 = df_zz.iloc[0, :-1].values.reshape(1, -1)

In [ ]:
df_z2 = df2[df2['Label'] == 2]
df_z2.iloc[4:10]
t2 = df_z2.iloc[:, :-1].values
print(t2)

In [ ]:
pred = xgb_scvic.predict(t2)

print(pred)


In [ ]:
import joblib

joblib.dump(xgb_scvic, 'xgb_svic.joblib')


# joblib.dump(rf_scvic, 'random_forest_scvic.pkl')
